# Sequence data

- 시퀀스 데이터는 순서가 있고 이것이 중요한 데이터로
- 자연어, 유전, 시계열 자료 등이 이런 자료형에 속한다.
  
    
- 모델링시 중요한 점은 이 순서의 의미를 반영해주어야 한다는 점이고
- 가장 각광받는 RNN과 CNN에 대해서 소개한다.

## RNN 

- 재귀적/순한 신경망을 뜻하며 데이터를 다룰 시에 히든 노드가 다음 히든 노드에 영향을 주는 모형
- 마찬가지로 역전파 알고리즘을 이용하여 업데이트하나 생각보다 코드 구현이 쉽지 않음(BPTT)
- 심플한 RNN보다는 LSTM이 엄청난 각광을 받았고 한국교수가 제안한 GRU도 유명함
- 코드 구성시 얼만큼의 시간을 고려할지를 인풋으로 넣어줘야함
    - function만들어서 가능
    - 자료형을 변형시켜서 가능
    

## CNN

- 본래 이미지를 대상으로하는 분석에서 엄청난 획을 그은 분석 기법
- 이미지관련 분석은 CNN전과 후로 나눌 수 있을 듯
- 시퀀스데이터에서도 비슷한 개념을 활요하여 접근 가능

In [1]:
from sklearn.datasets import load_boston
import numpy as np
import keras
from keras import models, layers
from keras.callbacks import EarlyStopping
import tensorflow as tf
tf.config.experimental.set_visible_devices([], 'GPU')

d = load_boston()

Using TensorFlow backend.


In [2]:
d.data[0]

array([6.320e-03, 1.800e+01, 2.310e+00, 0.000e+00, 5.380e-01, 6.575e+00,
       6.520e+01, 4.090e+00, 1.000e+00, 2.960e+02, 1.530e+01, 3.969e+02,
       4.980e+00])

In [3]:
temp_data = (d.data-np.mean(d.data))/np.std(d.data)

In [4]:
data = []
lags = 10

for i in range(lags, len(temp_data)):
    temp = temp_data[max(0, i-lags):i]
    data.append(temp)
    
data = np.array(data)

In [5]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

l11 = layers.SimpleRNN(32, return_sequences=False )(input_layer)
l12 = layers.LSTM(32, return_sequences=False)(input_layer)

l2 = layers.concatenate([l11, l12])

l3 = layers.Dense(32, activation='relu')(l2)

out_layer = layers.Dense(1, activation='linear')(l3)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10, 13)       0                                            
__________________________________________________________________________________________________
simple_rnn_1 (SimpleRNN)        (None, 32)           1472        input_1[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 32)           5888        input_1[0][0]                    
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 64)           0           simple_rnn_1[0][0]               
                                                                 lstm_1[0][0]               

In [7]:
# from keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [8]:


es = EarlyStopping(patience=20, monitor='val_loss', restore_best_weights=True)

In [9]:
model.fit(data, d.target[10:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es])

Train on 347 samples, validate on 149 samples
Epoch 1/400
347/347 [==============================] - 0s 1ms/step - loss: 636.8747 - mae: 23.7824 - val_loss: 268.8636 - val_mae: 14.2127
Epoch 2/400
347/347 [==============================] - 0s 181us/step - loss: 529.9627 - mae: 21.4115 - val_loss: 209.3344 - val_mae: 11.9480
Epoch 3/400
347/347 [==============================] - 0s 172us/step - loss: 425.4056 - mae: 18.8125 - val_loss: 145.9769 - val_mae: 9.0874
Epoch 4/400
347/347 [==============================] - 0s 178us/step - loss: 313.7905 - mae: 15.4402 - val_loss: 92.0389 - val_mae: 6.3819
Epoch 5/400
347/347 [==============================] - 0s 170us/step - loss: 200.2617 - mae: 11.4065 - val_loss: 67.1105 - val_mae: 5.5758
Epoch 6/400
347/347 [==============================] - 0s 172us/step - loss: 122.0687 - mae: 7.7977 - val_loss: 79.6038 - val_mae: 6.7603
Epoch 7/400
347/347 [==============================] - 0s 170us/step - loss: 82.4328 - mae: 6.1313 - val_loss: 113.478

In [10]:
model.evaluate(data, d.target[lags:])

496/496 [==============================] - 0s 56us/step


[126.17166660678002, 8.126870155334473]

In [11]:
keras.backend.clear_session()

input_layer = layers.Input(shape=(lags, d.data.shape[1]))

x1 = layers.Conv1D(filters=32, kernel_size=3, padding='same')(input_layer)
x1 = layers.MaxPooling1D()(x1)

x2 = layers.Conv1D(filters=16, kernel_size=5, padding='same')(input_layer)
x2 = layers.MaxPooling1D()(x2)

x = layers.concatenate([x1, x2])
x = layers.GlobalAveragePooling1D()(x)

x = layers.Dense(32, activation='relu')(x)

out_layer = layers.Dense(1, activation='linear')(x)

model = models.Model(input_layer, out_layer)

model.compile(optimizer='adam',
             loss = 'mse',
             metrics=['mae']
             )

In [12]:
data2 = np.reshape(data, (496, 10, 13))

In [13]:
model.fit(data2, d.target[lags:],
         epochs=400,
         validation_split=0.3,
         callbacks=[es]
         )

Train on 347 samples, validate on 149 samples
Epoch 1/400
347/347 [==============================] - 0s 391us/step - loss: 564.8226 - mae: 22.1919 - val_loss: 194.9921 - val_mae: 11.4103
Epoch 2/400
347/347 [==============================] - 0s 69us/step - loss: 398.9856 - mae: 18.0685 - val_loss: 102.5560 - val_mae: 7.1267
Epoch 3/400
347/347 [==============================] - 0s 66us/step - loss: 248.1104 - mae: 13.1849 - val_loss: 65.7516 - val_mae: 5.5931
Epoch 4/400
347/347 [==============================] - 0s 63us/step - loss: 128.5565 - mae: 8.1980 - val_loss: 121.8757 - val_mae: 9.2334
Epoch 5/400
347/347 [==============================] - 0s 60us/step - loss: 79.5043 - mae: 6.3645 - val_loss: 233.9686 - val_mae: 14.1256
Epoch 6/400
347/347 [==============================] - 0s 60us/step - loss: 77.3771 - mae: 7.1129 - val_loss: 252.9116 - val_mae: 14.7867
Epoch 7/400
347/347 [==============================] - 0s 60us/step - loss: 76.1644 - mae: 6.9204 - val_loss: 212.8170 - v

In [14]:
model.evaluate(data, d.target[lags:])

496/496 [==============================] - 0s 20us/step


[141.62040144397366, 8.810837745666504]